In [1]:
import pandas as pd
import numpy as np

In [2]:
index_name = "P0P4"
etf_name = "PGX"

# Output File Name
outputFile = '{} Rebalancing Numbers.xlsx'.format(etf_name)

# Total market Cap for PFF, needs to be manually updated
# Can be automated later
current_pgx_market_cap = 4892200000

# Reading ICE Data For Projected Universe
projected_universe_df = pd.read_excel('Data/P0P4-Projected.xlsx', skiprows=[0])

# Reading PFF Website Data for Current Shares
pgx_current_holdings_df = pd.read_excel('Data/PGX_Website_Data.csv')
pgx_current_holdings_df = pgx_current_holdings_df[['Security Identifier', ' Shares/Par Value']]
pgx_current_holdings_df['Security Identifier'] = pgx_current_holdings_df['Security Identifier'].astype(str)

# DataFrame for our final Output
pgx_df = pd.DataFrame()

In [3]:
# When reading data, below strings are read, they must be marked as np.nan
msg1 = 'Any unauthorized use or disclosure is prohibited. Nothing herein should in any way be deemed to alter the legal rights and obligations contained in agreements between any ICE Data Services entity ("ICE") and their clients relating to any of the Indices or products or services described herein. The information provided by ICE and contained herein is subject to change without notice and does not constitute any form of representation, or undertaking.  ICE and its affiliates make no warranties whatsoever, either express or implied, as to merchantability, fitness for a particular purpose, or any other matter in connection with the information provided. Without limiting the foregoing, ICE and its affiliates makes no representation or warranty that any information provided hereunder are complete or free from errors, omissions, or defects. All information provided by ICE is owned by or licensed to ICE. ICE retains exclusive ownership of the ICE Indices, including the ICE BofAML Indexes, and the analytics used to create this analysis ICE may in its absolute discretion and without prior notice revise or terminate the ICE information and analytics at any time. The information in this analysis is for internal use only and redistribution of this information to third parties is expressly prohibited.'
msg2 = 'Neither the analysis nor the information contained therein constitutes investment advice or an offer  or an invitation to make an offer  to buy or sell any securities or any options  futures or other derivatives related to such securities. The information and calculations contained in this analysis have been obtained from a variety of sources  including those other than ICE and ICE does not guarantee their accuracy.  Prior to relying on any ICE information and/or the execution of a security trade based upon such ICE information, you are advised to consult with your broker or other financial representative to verify pricing information. There is no assurance that hypothetical results will be equal to actual performance under any market conditions. THE ICE INFORMATION IS PROVIDED TO THE USERS "AS IS." NEITHER ICE, NOR ITS AFFILIATES, NOR ANY THIRD PARTY DATA PROVIDER WILL BE LIABLE TO ANY USER OR ANYONE ELSE FOR ANY INTERRUPTION, INACCURACY, ERROR OR OMISSION, REGARDLESS OF CAUSE, IN THE ICE INFORMATION OR FOR ANY DAMAGES RESULTING THEREFROM. In no event shall ICE or any of its affiliates, employees  officers  directors or agents of any such persons have any liability to any person or entity relating to or arising out of this information, analysis  or the indices  contained herein.'

In [4]:
# Projected Universe Data
projected_universe_df = projected_universe_df.replace('NaN', np.nan)
projected_universe_df = projected_universe_df.replace(msg1, np.nan)
projected_universe_df= projected_universe_df.replace(msg2, np.nan)

#drop row if ISIN number is Nan
projected_universe_df.dropna(subset=['ISIN number'], inplace=True)

In [5]:
projected_universe_df['modified_isin'] = projected_universe_df['ISIN number'].apply(lambda x: x[2:-1])

In [6]:
unique_ISIN = pd.concat([pgx_current_holdings_df['Security Identifier'], projected_universe_df['modified_isin']]).drop_duplicates().reset_index(drop=True)
pgx_df['ISIN number'] = unique_ISIN

In [7]:
isin_to_ticker_df = pd.read_excel('Static Data/ISINtoTicker.xlsx')
isin_to_ticker_df['modified_isin'] = isin_to_ticker_df['ISIN'].apply(lambda x: x[2:-1])
pgx_df = pgx_df.merge(isin_to_ticker_df, left_on='ISIN number', right_on='modified_isin', how='left').drop(['ISIN number'], axis=1)

In [8]:
pgx_df

,ISIN,Ticker,modified_isin
0,US48128B6487,JPM.PRC,48128B648
1,US48128B6552,JPM.PRD,48128B655
2,US94988U1512,WFC.PRZ,94988U151
3,US48128B5497,JPM.PRL,48128B549
4,US48128B5232,JPM.PRM,48128B523
...,...,...,...
327,BMG9078F1317,TRTN.PRB,G9078F131
328,US0844238701,WRB.PRH,084423870
329,US0844238883,WRB.PRG,084423888
330,US0844238057,WRB.PRF,084423805


In [9]:
pgx_df['Activ Ticker'] = pgx_df['Ticker'].str.replace(".PR","p", regex=False)

In [10]:
# getting the ActivDiv for reading the prices
activ_df = pd.read_excel('Static Data/ACtivDivData.xlsx', 'PasteByValues')
activ_df = activ_df[['Activ Ticker', 'Last Price']]

In [11]:
def getProjectedMarketCap(x):
    isid = x['ISIN']
    ticker = projected_universe_df[projected_universe_df['ISIN number']==isid]
    if(len(ticker)>0):
        return ticker.iloc[0]['% Mkt Value']
    return 0

pgx_df['Projected % Mkt Cap'] = pgx_df.apply(getProjectedMarketCap, axis=1)

In [12]:
def getLastPrice(x):
    activTicker = x['Activ Ticker']
    ticker = activ_df[activ_df['Activ Ticker']==activTicker]
    if(len(ticker)>0):
        return ticker.iloc[0]['Last Price']
    return np.nan

pgx_df['Last Price'] = pgx_df.apply(getLastPrice, axis=1)

In [13]:
def getCurrentPGXShares(x):
    misid = x['modified_isin']
    ticker = pgx_current_holdings_df[pgx_current_holdings_df['Security Identifier']==misid]
    if(len(ticker)>0):
        return ticker.iloc[0][' Shares/Par Value']
    return 0

pgx_df['Current PGX Shares'] = pgx_df.apply(getCurrentPGXShares, axis=1)

In [14]:
def getProjectedPGXShares(x):
    if(not isinstance(x['Last Price'], str)):
        return np.rint((current_pgx_market_cap*x['Projected % Mkt Cap'])/(x['Last Price']*100))
    return np.nan

pgx_df['Projected PGX Shares'] = pgx_df.apply(getProjectedPGXShares, axis=1)
pgx_df['Difference'] = pgx_df.apply(lambda x: x['Projected PGX Shares']-x['Current PGX Shares'], axis=1)

In [15]:
total_buys_ice = pgx_df['Difference'].where(pgx_df['Difference']>0).sum()
total_sells_ice = pgx_df['Difference'].where(pgx_df['Difference']<0).sum()
total_pgx_transactions_df = pd.DataFrame({"Number of Shares":[total_buys_ice, total_sells_ice]}, index=['Total PGX Buys', 'Total PGX Sells'])

In [16]:
with pd.ExcelWriter(outputFile, mode="a", if_sheet_exists="replace") as writer:
    pgx_df.to_excel(writer, sheet_name='PGX')
    total_pgx_transactions_df.to_excel(writer, sheet_name='Total Transactions - PGX')

In [17]:
pgx_df

,ISIN,Ticker,modified_isin,Activ Ticker,Projected % Mkt Cap,Last Price,Current PGX Shares,Projected PGX Shares,Difference
0,US48128B6487,JPM.PRC,48128B648,JPMpC,1.67367,25.33,3108286,9057565.0,5949279.0
1,US48128B6552,JPM.PRD,48128B655,JPMpD,1.50731,24.88,2890669,8304797.0,5414128.0
2,US94988U1512,WFC.PRZ,94988U151,WFCpZ,1.43254,19.93,3550113,9853177.0,6303064.0
3,US48128B5497,JPM.PRL,48128B549,JPMpL,1.35651,20.53,3323872,9057553.0,5733681.0
4,US48128B5232,JPM.PRM,48128B523,JPMpM,1.34863,18.88,3520629,9791915.0,6271286.0
...,...,...,...,...,...,...,...,...,...
327,BMG9078F1317,TRTN.PRB,G9078F131,TRTNpB,0.13123,25.56,0,703800.0,703800.0
328,US0844238701,WRB.PRH,084423870,WRBpH,0.20090,18.75,0,1468776.0,1468776.0
329,US0844238883,WRB.PRG,084423888,WRBpG,0.17081,19.13,0,1223982.0,1223982.0
330,US0844238057,WRB.PRF,084423805,WRBpF,0.23412,21.85,0,1468804.0,1468804.0
